In [11]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from multiprocessing import Pool
products = pd.read_csv(
    'products.csv',
    sep=';'
)

In [12]:
#create a dictionnary of exchange rate, base currency = Eur
url_currency_euro_rate= 'https://api.exchangerate-api.com/v4/latest/EUR'
res2= requests.get(url_currency_euro_rate)
currencies_to_eur = res2.json()['rates']

In [13]:
restcountries_data = requests.get('https://restcountries.eu/rest/v2/all').json()

In [14]:
countrycode_to_currency = {
    country['alpha2Code']: country['currencies'][0]['code']
    for country in restcountries_data
}

countrycode_to_currency = {}
for country in restcountries_data:
    country_code = country['alpha2Code']
    currency = country['currencies'][0]['code']
    countrycode_to_currency[country_code] = currency

In [15]:
def ip_to_countrycode(ip):
    url = f'http://api.ipstack.com/{ip}'
    IPSTACK_KEY = 'd305c0652f17a34f0bfdf555f5b42cfe'
    data = requests.get(url, params={'access_key': IPSTACK_KEY}).json()
    return data['country_code']

In [ ]:
ip_ok = products.ip_address.str.contains(
    '^(([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])\.){3}([0-9]|[1-9][0-9]|1[0-9]{2}|2[0-4][0-9]|25[0-5])$'
)

codes = [ip_to_countrycode(ip) for ip in products[ip_ok].ip_address]
products.loc[ip_ok, 'country_code'] = codes